# Raffiot functions for project 1

Usefull imports

In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

Helpers

In [3]:
from proj1_helpers import load_csv_data, predict_labels, create_csv_submission
from helpers import *

cheminTrain = 'C:/Users/Raphael/Desktop/EPFL/Machine_Learning/train.csv'
cheminTest =  'C:/Users/Raphael/Desktop/EPFL/Machine_Learning/test.csv'

def standardize(x):
    """Standardize the original data set."""
    mean_x = np.mean(x, axis=0)
    x = x - mean_x
    std_x = np.std(x, axis=0)
    x = x / std_x
    return x, mean_x, std_x

# load data.
yb, input_data, ids = load_csv_data(cheminTrain,True)
yb = (yb+1)/2
yb_test, input_data_test, ids_test = load_csv_data(cheminTest)

# build sampled x and y.
x, mean_x, std_x = standardize(input_data)
x_test, meanx_test, std_x_test = standardize(input_data_test)


1. Loss functions & compute gradient function

In [4]:
import numpy as np

def compute_mse(y, tx, w):
    return (1/(2*tx.shape[0]))*np.linalg.norm(y-tx@w)**2
def compute_loss_rmse(y,tx,w):
    return (compute_mse(y,tx,w)*2)**0.5
def compute_gradient_LS(y, tx, w):
    return -(1/tx.shape[0])*np.transpose(tx)@(y-tx@w)
def sigmoid(t):
    return np.exp(t)/(1+np.exp(t))
def calculate_loss_LR(y, tx, w):
    sum = 0
    for i in range(y.shape[0]):
        sum += np.log(1+np.exp(tx[i].transpose()@w))-y[i]*(tx[i].transpose()@w)
    return sum
def calculate_gradient_LR(y, tx, w):
    return tx.transpose()@(sigmoid(tx@w)-y)
def calculate_loss_RLR(y, tx, w, lambda_):
    return calculate_loss_LR(y,tx,w) + (lambda_/2)*sum(w**2)
def calculate_gradient_RLR(y, tx, w, lambda_):
    return calculate_gradient_LR(y,tx,w) + lambda_*np.sum(w)

 2- Linear regression using gradient descent

In [5]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    
    w = initial_w
    loss=0
    for n_iter in range(max_iters):
        loss = compute_mse(y,tx,w)
        gradient = compute_gradient_LS(y,tx,w)
        w = w - gamma*gradient

    
    return loss, w

 3- Linear regression using stochastic gradient descent

In [6]:
def stochastic_gradient_descent(y, tx, initial_w, batch_size, max_iters, gamma):
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            loss = compute_mse(minibatch_y,minibatch_tx,w)
            gradient = compute_gradient_LS(minibatch_y,minibatch_tx,w)
            
            w = w - gamma*gradient
            ws.append(w)
            losses.append(loss)
 
    indexMin = values.index(min(losses))
    
    return losses[indexMin], ws[indexMin]

 4- Least squares regression using normal equations

In [7]:
def least_squares(y, tx):
    txt = tx.transpose()
    w= np.linalg.solve(txt@tx,txt@y)
    loss = compute_mse(y,tx,w)
    
    return loss, w

 5- Ridge regression using normal equations

In [8]:
def ridge_regression(y, tx, lambda_):
    
    txt = tx.transpose()
    lambda_prim = 2*y.shape[0]*lambda_
    identity = np.identity(tx.shape[1])
    w = np.linalg.inv(txt@tx+lambda_prim*identity)@txt@y
    #w= np.linalg.solve(txt@tx+lambda_prim*identity,txt@y)
    loss = compute_mse(y,tx,w)
    return loss,w

6- Logistic regression using gradient descent or SGD

In [9]:
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    w = initial_w
    loss = 0
    for iter in range(max_iters):
        loss = calculate_loss_LR(y,tx,w)
        gradient = calculate_gradient_LR(y,tx,w)
        w= w - gamma*gradient
    return loss, w

7- Regularized logistic regression using gradient descent or SGD

In [10]:
def reg_logistic_regression(y, tx, lambda_ ,initial_w, max_iters, gamma):
    w = initial_w
    for iter in range(max_iters):
        
        loss = calculate_loss_RLR(y,tx,w,lambda_)
        gradient = calculate_gradient_RLR(y,tx,w,lambda_)
        print("iteration={i}, loss={l}, w0={w0}, w1={w1}, w2={w2}".format(i=iter,l=loss,w0=w[0],w1=w[1],w2=w[2]))
        print("iteration={i} g0={g0}, g1={g1} ,g2={g2}".format(i=iter,g0=gradient[0],g1=gradient[1],g2=gradient[2]))
        w = w - gamma*gradient
    loss = calculate_loss_RLR(y,tx,w,lambda_)
    return loss,w

In [11]:
initial_w = np.ones((x.shape[1],))*0.001
gamma =0.01
max_iters = 100
lambda_ = 0.01

#loss,w = ridge_regression(y, tx, lambda_)


#y_pred = predict_labels(w, x_test)
#create_csv_submission(ids_test, y_pred, 'submission_for_kaggle.csv')